## Importing Libraries

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from bs4 import BeautifulSoup
import tf_keras as keras
import re
import requests

## Testing with a single URL

In [2]:
sentiment_analysis = pipeline("sentiment-analysis")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

C:\Users\79860\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\79860\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
url = "https://finance.yahoo.com/news/china-pboc-keeps-gold-buying-021556867.html"
r = requests.get(url) #Makes request to get information from the URL
soup = BeautifulSoup(r.text, 'html.parser') #Parses the HTML content of the URL
para = soup.find_all('p') #Finds all the paragraphs in the HTML content

In [ ]:
text = [paragraph.text for paragraph in para] #Extracts the text from the paragraphs
words = ' '.join(text).split(' ')[:400] #Grabs the first 400 words of the article for the model to summarize
article = ' '.join(words) #Joins the words together to form the article

In [ ]:
article

In [ ]:
input_ids = tokenizer.encode(article, return_tensors='pt') #Encodes the article into input_ids
output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True) #Generates the summary
summary = tokenizer.decode(output[0], skip_special_tokens=True) #Decodes the summary
summary